In [2]:
import pandas as pd

In [52]:
df = pd.read_parquet('../../data/curated/2022_vote_probs')

In [53]:
df

,match_id,match_round,player_first_name,player_last_name,player_team,0,1,2,3,predictions
player_id,,,,,,,,,,
11904,16117,1,Tom,Liberatore,Western Bulldogs,0.995934,0.002583,0.001251,0.000232,0.0
11945,16117,1,Steven,May,Melbourne,0.992242,0.005301,0.002085,0.000373,0.0
11972,16117,1,Max,Gawn,Melbourne,0.929986,0.032486,0.030978,0.006549,0.0
12015,16117,1,Tom,McDonald,Melbourne,0.992080,0.004390,0.003016,0.000514,0.0
12034,16117,1,Adam,Tomlinson,Melbourne,0.995563,0.002950,0.001256,0.000231,0.0
...,...,...,...,...,...,...,...,...,...,...
12863,16346,23,Logan,McDonald,Sydney,0.990389,0.005911,0.003013,0.000687,0.0
12864,16346,23,Errol,Gulden,Sydney,0.981063,0.011582,0.005983,0.001371,0.0
12945,16346,23,Mitch,Owens,St Kilda,0.992731,0.004832,0.002021,0.000416,0.0


In [54]:
match_ids = df.match_id.unique()

In [55]:
def standardise_votes(df):
    
    ini_sum_1 = sum(df['1'])
    ini_sum_2 = sum(df['2'])
    ini_sum_3 = sum(df['3'])
    
    df['new_3'] = df['3'] / ini_sum_3
    df['new_2'] = df['2'] / ini_sum_2
    df['new_1'] = df['1'] / ini_sum_1

    df['new_0'] = 1 - (df['new_1'] + df['new_2'] + df['new_3'])
    
    df['exp_votes'] = 1*df['new_1'] + 2*df['new_2'] + 3*df['new_3']
    
    
    return df[['player_id', 'match_id', 'player_first_name', 'player_last_name', 'player_team', 'new_0', 'new_1', 'new_2', 'new_3', 'exp_votes']]

In [56]:
final_df = pd.DataFrame(columns=['player_id', 'match_id', 'player_first_name', 'player_last_name', 'player_team', 'new_0', 'new_1', 'new_2', 'new_3', 'exp_votes'])


for idx in match_ids:
    
    df_match = df[df['match_id'] == idx].reset_index() 
    
    final_df = pd.concat([final_df, standardise_votes(df_match)])

In [57]:
final_df

,player_id,match_id,player_first_name,player_last_name,player_team,new_0,new_1,new_2,new_3,exp_votes
0,11904,16117,Tom,Liberatore,Western Bulldogs,0.996153,0.002482,0.001176,0.000190,0.005403
1,11945,16117,Steven,May,Melbourne,0.992642,0.005093,0.001960,0.000305,0.009927
2,11972,16117,Max,Gawn,Melbourne,0.934318,0.031216,0.029109,0.005358,0.105507
3,12015,16117,Tom,McDonald,Melbourne,0.992527,0.004218,0.002834,0.000421,0.011148
4,12034,16117,Adam,Tomlinson,Melbourne,0.995796,0.002834,0.001181,0.000189,0.005762
...,...,...,...,...,...,...,...,...,...,...
41,12863,16346,Logan,McDonald,Sydney,0.991393,0.005060,0.002746,0.000801,0.012956
42,12864,16346,Errol,Gulden,Sydney,0.983033,0.009915,0.005452,0.001599,0.025619
43,12945,16346,Mitch,Owens,St Kilda,0.993537,0.004136,0.001841,0.000486,0.009276
44,12947,16346,Nasiah,Wanganeen-Milera,St Kilda,0.992664,0.004435,0.002318,0.000583,0.010820


In [58]:
final_df.to_parquet('../../data/curated/adjusted_probabilities_2022')